

# Install Library


In [ ]:
!pip install nltk
!pip install Sastrawi
!pip install regex
!pip install tweet-preprocessor
!pip install swifter

     |████████████████████████████████| 634kB 6.9MB/s 
     |████████████████████████████████| 296kB 12.9MB/s 
     |████████████████████████████████| 583kB 14.6MB/s 
     |████████████████████████████████| 112kB 27.8MB/s 
     |████████████████████████████████| 48.5MB 85kB/s 
     |████████████████████████████████| 17.2MB 259kB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 133kB 47.4MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
     |████████████████████████████████| 3.1MB 33.1MB/s 
     |████████████████████████████████| 1.3MB 34.4MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 143kB 49.7MB/s 
     |████████████████████████████████| 296kB 40.1MB/s 
  Created wheel for swifter: filename=swifter-1.0.7-cp37-none-any.whl size=13980 sha256=c5ab5f49618ce33a7e4125

# Import Library

In [ ]:
#pandas and numpy
import pandas as pd
import numpy as np

#nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords

#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import swifter

#regex
import string
import re

#tweet preprocessorF
import preprocessor  as p

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Import Data Tweet

In [ ]:
df = pd.read_csv('/content/politik.csv')

# Prerocessing

hapus tweet selain bahasa indonesia

In [ ]:
df = df[df['language'] == 'id'].reset_index.drop(columns='index')

Case Folding

In [ ]:
df['case folded tweet'] = df['tweet'].str.lower()

Normalization

Remove Mention, Hashtag, Emoji, Smiley, Number, URL

In [ ]:
def normalization(tweet):
    p.set_options(p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER, p.OPT.SMILEY, p.OPT.URL)
    text = p.clean(tweet)
    return text

In [ ]:
#using tweet_preprocessor library
df['tweet_normalized'] = df['case folded tweet'].apply(normalization)

In [ ]:
#using regex

#remove special char
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['tweet_normalized'] = df['tweet_normalized'].apply(remove_tweet_special)
df['tweet_normalized'] = df['tweet_normalized'].apply(remove_number)
df['tweet_normalized'] = df['tweet_normalized'].apply(remove_punctuation)
df['tweet_normalized'] = df['tweet_normalized'].apply(remove_whitespace_multiple)
df['tweet_normalized'] = df['tweet_normalized'].apply(remove_single_char)

Tokenizing (NLTK)

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tweet_tokenized'] = df['tweet_normalized'].apply(word_tokenize_wrapper)

Normalisasi kata gaul dan singakatan

In [ ]:
kamus_alay = pd.read_csv('/content/new_kamusalay.csv', encoding='latin-1')

kamus_alay_dict = {}

for index, row in kamus_alay.iterrows():
  if row[0] not in kamus_alay:
    kamus_alay_dict[row[0]] = row[1]

def normalized_alay(document):
  return [kamus_alay_dict[term] if word in kamus_alay_dict else word for word in document]

df['tweet_normalized_slang'] = df['tweet_tokenized'].apply(normalized_alay)  

Filtering

In [ ]:
#ambil stopword bahasa indonesia
list_stopwords = stopwords.words('indonesian')

#menambahkan stopword secara manual
list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'no', 'je', 'om', 'pru', 'sch',
                       'injirrr', 'ah', 'oena', 'bu', 'eh'])

list_stopwords = set(list_stopwords)

def stopwords_removal(document):
  return [word for word in document if word not in list_stopwords]

df['filtered_tweet'] = df['tweet_normalized_slang'].apply(stopwords_removal)

Stemming

In [ ]:
#buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stemmed wrapper
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in df['filtered_tweet']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

#memmulai stemming
def apply_stemmed_term(document):
  return [term_dict[term] for term in document]

df['stemmed_tweet'] = df['filtered_tweet'].swifter.apply(apply_stemmed_term)

441
------------------------
kepala : kepala
biro : biro
politik : politik
hamas : hamas
ismail : ismail
haniah : haniah
sujud : sujud
syukur : syukur
kemenangan : menang
pejuang : juang
palestina : palestina
intelijen : intelijen
aktif : aktif
bermain : main
media sosial : media sosial
berita : berita
editan : edit
media : media
opini : opini
akun : akun
pendukung : dukung
warganet : warganet
terprovokasi : provokasi
hina : hina
omong : omong
sembarangan : sembarang
dipertaruhkan : taruh
bahas : bahas
utamakan : utama
domestik : domestik
prospek : prospek
kerja : kerja
ilmu : ilmu
komunikasi : komunikasi
masuk : masuk
dunia : dunia
mencari : cari
bingung : bingung
setuju : tuju
amhendropriyono : amhendropriyono
urusan : urus
mengurus : urus
kehidupan : hidup
anak : anak
bagus : bagus
gen : gen
penerus : terus
pekerjaan rumah : kerja rumah
konflik : konflik
lawan : lawan
wilayah : wilayah
agama : agama
stop : stop
framing : framing
isu : isu
aturan : atur
dimana : mana
area : area
hala

In [ ]:
df['stemmed_tweet']

0     [kepala, biro, politik, hamas, ismail, haniah,...
1     [intelijen, aktif, main, media sosial, berita,...
5     [prospek, kerja, ilmu, komunikasi, masuk, duni...
6     [tuju, amhendropriyono, urus, urus, hidup, ana...
8     [atur, mana, area, halaman, duta, bera, daulat...
9     [derma, derma, ahli, politik, suka, main, derm...
11    [politik, malaysia, perintah, perosak, dajal, ...
12    [kadrun, pura, zolimin, tindas, intimidasi, di...
13         [menteri, nak, politik, lebai, pon, bangsat]
16    [point, politik, takut, kalah, kalah, tulah, m...
17       [akibat, mudah, akun, bajak, penting, politik]
20    [belah, baik, sengaja, biar, taplak, meja, iko...
23    [urus, ibadah, wajib, individu, jual, politik,...
24         [juang, politik, koridor, konstitusi, keras]
25    [sandiwara, politik, muslim, negara, muslim, z...
28    [gampang, banget, maki, kadrun, isu, geopoliti...
32    [lihat, beda, humanitarian, crisis, gaza, bang...
33    [tuntut, kerumun, surat pemberitahuan tahu

In [ ]:
df['label'] = 'politik'

In [ ]:
df.to_csv('Sample Preprocessing.csv')

In [ ]:
df[['stemmed_tweet', 'label']].to_csv('Sample Bahan TF-IDF.csv')